In [38]:
import csv
import itertools
import json
import utils
import random

from collections import defaultdict

In [39]:
with open("../data/nana-naba-verbs_nk_240407.json", "r") as f:
    data = json.load(f)

# data
# verbs = defaultdict(list)
verbs = []
for k, v in data.items():
    for verb, items in v.items():
        verbs.append(verb)

themes = set()
for k, v in data.items():
    for verb, items in v.items():
        for t in items['theme']:
            themes.add(t)

recipients = set()
for k, v in data.items():
    for verb, items in v.items():
        for r in items['recipient']:
            recipients.add(r)

In [40]:
aochildes = utils.read_file("../../smolm/data/corpora/babylm_data/babylm_100M/aochildes.train")

verb_sents = defaultdict(list)
for sent in aochildes:
    for verb in verbs:
        if verb in sent:
            verb_sents[verb].append(sent)
verb_sents = dict(verb_sents)

In [41]:
AGENTS = ["grandma", "ryan", "ethan", "john", "nina", "lily", "adam", "david", "you", "the boy", "he", "she", "i", "mommy", "daddy", "laura"]

# "|".join(AGENTS)
verb_agents = defaultdict(list)
for verb in verbs:
    for a in AGENTS:
        query = f"{a} {verb}"
        found = False
        try:
            for sent in verb_sents[verb]:
                if query in sent:
                    found = True
        except:
            pass
        if not found:
            verb_agents[verb].append(a)
verb_agents = dict(verb_agents)

In [42]:
random.seed(42)

naba_nana = []
verb_id = 1
item_id = 1
for k, v in data.items():
    for verb, items in v.items():
        # verbs.append(verb)
        for theme, recipient in itertools.product(items["theme"], items["recipient"]):
            agent_space = [
                a for a in verb_agents[verb] if a != theme and a != recipient
            ]
            agent = random.choice(agent_space)
            do = f"{agent} {verb} {recipient} {theme} ."
            pp = f"{agent} {verb} {theme} to {recipient} ."
            naba_nana.append([item_id, verb_id, k, verb, do, pp])
            item_id += 1
        verb_id += 1

In [44]:
# write to csv
with open("../data/naba-nana-sentences.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["item_id", "verb_id", "verb_type", "verb", "do", "pp"])
    for row in naba_nana:
        writer.writerow(row)